# NMF Topic Modeling

Use BII, 20 cores, 3 hours, 100 Ram.

In [1]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])?  y


In [1]:
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
import gensim
import time

from sklearn.decomposition import NMF, TruncatedSVD, LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from gensim.models.coherencemodel import CoherenceModel
import ast

In [3]:
#TODO:Input a dataframe that are ai related abstracts, need variables: final_frqwds_removed
#abstracts = pd.read_csv(r'/home/zz3hs/git/dspg21RnD/data/dspg21RnD/bert_ai_abstracts_2.csv') 



In [4]:
abstracts["final_frqwds_removed"]

0       ['direct', 'identification', 'basic', 'apply',...
1       ['previous', 'grant', 'award', 'ies', 'nationa...
2       ['previous', 'grant', 'award', 'ies', 'graspab...
3       ['team', 'prototype', 'web', 'coding', 'progre...
4       ['prototype', 'science', 'quest', 'science', '...
                              ...                        
7653    ['artificial_intelligence', 'fuel', 'recent', ...
7654    ['frontier', 'trustworthy', 'machine_learning'...
7655    ['predictive', 'underdamped', 'pneumatically_a...
7656    ['order', 'optimal', 'value', 'decision', 'ani...
7657    ['american_indian', 'science', 'engineering', ...
Name: final_frqwds_removed, Length: 7658, dtype: object

In [24]:
#pd.set_option("display.max_colwidth", -1)
#abstracts[abstracts["cosine_similarity_score"] >0.55].ABSTRACT

In [25]:
#abstracts[abstracts["cosine_similarity_score"]  <0.41].ABSTRACT

# Coherence Model to find the optimal number of topics for NMF

In [5]:
docs = abstracts["final_frqwds_removed"]

clean_docs = []
for doc in docs:
    doc = ast.literal_eval(doc)
    clean_docs.append(doc)
    
text = []
for abstract in clean_docs:
    text.append(" ".join(abstract))

In [11]:
clean_docs[0]

['direct',
 'identification',
 'basic',
 'apply',
 'scientific',
 'implementation',
 'ensure',
 'optimum',
 'management',
 'forest',
 'administration',
 'scientist',
 'discipline']

In [7]:
text[0]

'direct identification basic apply scientific implementation ensure optimum management forest administration scientist discipline'

In [8]:
tfidf_vectorizer = TfidfVectorizer(max_df=1.0, min_df=3)
tf_idf = tfidf_vectorizer.fit_transform(text)

In [9]:
#make sure the number of terms (column) is less the 100,000
#if greater than 100,000, use the number of terms as the keep_n parameter
tf_idf.shape

(7658, 10966)

In [10]:
def createVars(docs):
    # Create the variables needed for NMF from df[final_frqwds_removed]: dictionary (id2word), corpus
    
    # Create Dictionary
    id2word = gensim.corpora.Dictionary(docs)
    id2word.filter_extremes(no_below=3, no_above=1.0,  keep_n = 100000)
    
    # Create Corpus (Term Document Frequency)
    #Creates a count for each unique word appearing in the document, where the word_id is substituted for the word
    corpus = [id2word.doc2bow(doc) for doc in docs]

    return id2word, corpus

In [12]:
#TODO: run the following code to generate id2word and corpus
id2word, corpus = createVars(clean_docs)
#TODO: RENAME the file, run the code to save the output
docs  = clean_docs
#pickle.dump([corpus, id2word, docs], open('../../data/dspg21RnD/coherence_vars_bert2.sav','wb'))

Use keep_n = 100,000 defalt.


# Read in Coherence file

In [2]:
#TODO: Read in your coherence data (change the name of the file)
f = open('../../data/dspg21RnD/coherence_vars_bert2.sav', 'rb')
[corpus, id2word, docs] = pickle.load(f)
f.close()

In [3]:
len(docs)

7658

In [4]:
docs[0]

['direct',
 'identification',
 'basic',
 'apply',
 'scientific',
 'implementation',
 'ensure',
 'optimum',
 'management',
 'forest',
 'administration',
 'scientist',
 'discipline']

In [5]:
text = []
for doc in docs:
    text.append(" ".join(doc))

In [6]:
text[0]

'direct identification basic apply scientific implementation ensure optimum management forest administration scientist discipline'

In [7]:
# function slightly modified from https://nlpforhackers.io/topic-modeling/

def print_topics(model, vectorizer, top_n=10):
    for idx, topic in enumerate(model.components_):  # loop through each row of H.  idx = row index.  topic = actual row
        print("\nTopic %d:" % (idx))
        #print([(vectorizer.get_feature_names()[i], topic[i])  # printing out words corresponding to indices found in next line
                        #for i in topic.argsort()[:-top_n - 1:-1]])  # finding indices of top words in topic
            
        print_list = [(vectorizer.get_feature_names()[i], topic[i])  
                        for i in topic.argsort()[:-top_n - 1:-1]]
        for item in print_list:
            print(item)
            
# Function to format topics as a "list of list of strings".
# Needed for topic coherence function in Gensim

# function modified from https://nlpforhackers.io/topic-modeling/

def list_topics(model, vectorizer, top_n=10):

    #input. top_n: how many words to list per topic.  If -1, then list all words.
       
    topic_words = []
    
    for idx, topic in enumerate(model.components_):  # loop through each row of H.  idx = row index.  topic = actual row
            
        if top_n == -1:   
            topic_words.append([vectorizer.get_feature_names()[i] for i in topic.argsort()[::-1]])
        else:
            topic_words.append([vectorizer.get_feature_names()[i] for i in topic.argsort()[:-top_n - 1:-1]])
        
    return topic_words

In [8]:
# create document-term matrix - TFIDF 
tfidf_vectorizer = TfidfVectorizer(max_df=1.0, min_df=3)
tf_idf = tfidf_vectorizer.fit_transform(text)

In [9]:
# function adapted from https://datascienceplus.com/evaluation-of-topic-modeling-topic-coherence/

def nmf_metrics(doc_term_matrix, n_topics, vectorizer, corpus, id2word, docs, rand_start):
    """
    Compute c_v topic coherence for various number of topics
    Parameters:
    ----------
    tf_idf
    n_topics : list of number of topics
    Returns:
    -------
    coherence_values : c_v topic coherence values corresponding to the NMF model with respective number of topics
    """
    
    coherence_values = []
    
    i = rand_start
    for num_topics in n_topics:

        # create model
        t1 = time.time()
        nmf_model = NMF(n_components=num_topics, random_state = i)
        nmf_model.fit_transform(doc_term_matrix)
        t2 = time.time()
        print(f"  Model time: {t2-t1}")
        
        # create list of topics
        topics = list_topics(nmf_model, vectorizer, top_n=10)
        
        # calculate coherence
        t1 = time.time()
        
        #TODO:manually adjust number of processes
        cm = CoherenceModel(topics=topics, corpus=corpus, dictionary=id2word, texts=docs, 
                            coherence='c_v', #model for calculating coherence score
                            processes=10 #for smaller corpus, pronesses= number of cores - 5
                           ) #window_size=500 ) 
        coherence_values.append(cm.get_coherence())
        t2 = time.time()
        print(f"  Coherence time: {t2-t1}")
        
        # output completion message
        i = i+1
        print('Number of topics =', num_topics, "complete.")

    return coherence_values


In [ ]:
# code copied from https://datascienceplus.com/evaluation-of-topic-modeling-topic-coherence/
# minor alterations made
n_topics = list(range(5,51,5)) + [75,100,125] #takes about 4-5 hrs to run

num_runs = 10

col_names = [f"iteration {i}" for i in range(num_runs)]
nmf_c = pd.DataFrame(index = n_topics, columns = col_names)

for i in range(num_runs):
    print(f"Iteration {i}")
    
    # run models
    c = nmf_metrics(doc_term_matrix=tf_idf, n_topics=n_topics, vectorizer=tfidf_vectorizer, 
                         corpus=corpus, id2word=id2word, docs=docs, rand_start = (i)*len(n_topics))
    
    # save results
    nmf_c[f"iteration {i}"] = c

Iteration 0
  Model time: 1.1127963066101074
  Coherence time: 0.6914005279541016
Number of topics = 5 complete.
  Model time: 1.2639811038970947
  Coherence time: 0.9571599960327148
Number of topics = 10 complete.


/home/zz3hs/.conda/envs/crystal_bert/lib/python3.8/site-packages/sklearn/decomposition/_nmf.py:1076: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn("Maximum number of iterations %d reached. Increase it to"


  Model time: 3.973792791366577
  Coherence time: 1.2598192691802979
Number of topics = 15 complete.
  Model time: 3.356252908706665
  Coherence time: 1.4911510944366455
Number of topics = 20 complete.


/home/zz3hs/.conda/envs/crystal_bert/lib/python3.8/site-packages/sklearn/decomposition/_nmf.py:1076: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn("Maximum number of iterations %d reached. Increase it to"


  Model time: 6.984228134155273
  Coherence time: 2.0317554473876953
Number of topics = 25 complete.
  Model time: 7.8718883991241455
  Coherence time: 2.1873152256011963
Number of topics = 30 complete.
  Model time: 5.267805099487305
  Coherence time: 2.533888816833496
Number of topics = 35 complete.
  Model time: 8.182427167892456
  Coherence time: 2.9799013137817383
Number of topics = 40 complete.
  Model time: 6.447145938873291
  Coherence time: 3.2120890617370605
Number of topics = 45 complete.
  Model time: 11.465293169021606
  Coherence time: 3.6982080936431885
Number of topics = 50 complete.


...

  Model time: 4.084392547607422
  Coherence time: 1.2412874698638916
Number of topics = 15 complete.
  Model time: 3.91092848777771
  Coherence time: 1.515458583831787
Number of topics = 20 complete.
  Model time: 5.591097354888916
  Coherence time: 1.831383466720581
Number of topics = 25 complete.
  Model time: 5.612567663192749
  Coherence time: 2.3275649547576904
Number of topics = 30 complete.
  Model time: 8.965773582458496
  Coherence time: 2.5669238567352295
Number of topics = 35 complete.
  Model time: 6.586566686630249
  Coherence time: 2.848316192626953
Number of topics = 40 complete.
  Model time: 12.149762392044067
  Coherence time: 3.610668420791626
Number of topics = 45 complete.


/home/zz3hs/.conda/envs/crystal_bert/lib/python3.8/site-packages/sklearn/decomposition/_nmf.py:1076: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn("Maximum number of iterations %d reached. Increase it to"


  Model time: 17.96056032180786
  Coherence time: 3.768014907836914
Number of topics = 50 complete.
  Model time: 33.52816104888916
  Coherence time: 6.0313026905059814
Number of topics = 75 complete.
  Model time: 48.196704149246216
  Coherence time: 7.966750621795654
Number of topics = 100 complete.
  Model time: 68.61280179023743
  Coherence time: 10.46601676940918
Number of topics = 125 complete.
Iteration 5
  Model time: 0.795116662979126
  Coherence time: 0.6755280494689941
Number of topics = 5 complete.
  Model time: 1.3044605255126953
  Coherence time: 0.9600212574005127
Number of topics = 10 complete.
  Model time: 1.112351655960083
  Coherence time: 1.291261911392212
Number of topics = 15 complete.
  Model time: 4.857828617095947
  Coherence time: 1.7780160903930664
Number of topics = 20 complete.


/home/zz3hs/.conda/envs/crystal_bert/lib/python3.8/site-packages/sklearn/decomposition/_nmf.py:1076: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn("Maximum number of iterations %d reached. Increase it to"


  Model time: 7.3296802043914795
  Coherence time: 1.9288251399993896
Number of topics = 25 complete.
  Model time: 6.608461380004883
  Coherence time: 2.131995677947998
Number of topics = 30 complete.
  Model time: 4.594269037246704
  Coherence time: 2.510430097579956
Number of topics = 35 complete.
  Model time: 7.561211109161377
  Coherence time: 3.061439275741577
Number of topics = 40 complete.
  Model time: 10.162342548370361
  Coherence time: 3.208566427230835
Number of topics = 45 complete.
  Model time: 10.863919973373413
  Coherence time: 3.844336986541748
Number of topics = 50 complete.


In [ ]:
# TODO: save results as csv, no need to save as pickle file anymore, change file name
nmf_c['n_topics'] = nmf_c.index
#nmf_c.to_pickle("/home/zz3hs/git/dspg21RnD/data/dspg21RnD/nmf_coherence_XXX.pkl")
nmf_c.to_csv("/home/zz3hs/git/dspg21RnD/data/dspg21RnD/nmf_coherence_xxx.csv", index= False)